In [2]:
import logging
logging.getLogger().setLevel(logging.WARNING)

from konlpy.corpus import kobill, kolaw
from konlpy.tag import Twitter
twitter = Twitter()

stopwords = "수 년 등 및 몇 중 네이버 뉴스".split()
stopwords += "공급 설치 조성 운영 실행 설립 확대 제공 실시 지원 검토 육성 추진 유치 강화 개선 구축 마련 확충 실시 개선 해소".split()
stopwords = set(stopwords)

pos = lambda d: ['/'.join(p) for p in twitter.pos(d, stem=True, norm=True) if p[0] not in stopwords and p[1] in ('Noun', 'Verb', 'Alpha', 'Adjective')]

In [3]:
twitter.pos('청각장애인과 함께 CCTV로 만드는 안전한 서울', stem=True, norm=True)

[('청각장애', 'Noun'),
 ('인과', 'Noun'),
 ('함께', 'Adverb'),
 ('CCTV', 'Alpha'),
 ('로', 'Noun'),
 ('만들다', 'Verb'),
 ('안전하다', 'Adjective'),
 ('서울', 'Noun')]

In [4]:
%%time
texts_kobill = [pos(kobill.open(i).read()) for i in kobill.fileids()]
texts_kolaw = [pos(kolaw.open(i).read()) for i in kolaw.fileids()]

CPU times: user 7.31 s, sys: 172 ms, total: 7.48 s
Wall time: 3.55 s


In [107]:
%%time
import json

texts = []

for fname in ('jisa_tagged.json', 'mayor_tagged.json',):
    with open(fname) as data_file:    
        data = json.load(data_file)
        for voting_district in data:
            if voting_district['city'] != 'seoul' or voting_district['district'] > 25:
                continue
            promises = voting_district['promises']
            for p in promises:
                texts.append((p['title'], pos(p['title']), (voting_district['city'], voting_district['district'])))

texts_promises = [text[1] for text in texts]
print(len(texts_promises))

1328
CPU times: user 1.27 s, sys: 801 ms, total: 2.08 s
Wall time: 2.94 s


In [108]:
set([d[2][1] for d in texts])

{0,
 1,
 2,
 3,
 4,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 23,
 24,
 25}

In [109]:
from gensim import corpora, models
# Create dictionary
corpus = texts_kobill + texts_kolaw + texts_promises
#corpus = texts_promises
logging.info('Dictionary from %d documents' % len(corpus))
dictionary_ko = corpora.Dictionary(corpus)
# Filter terms that occur in more than 2% of the documents
dictionary_ko.filter_extremes(no_below=0, no_above=0.02)
# Save for later use
dictionary_ko.save('ko.dict')

In [110]:
%%time
from gensim import similarities

num_topics = 50

# Generate Lsi model for promise text corpus
corpus = [dictionary_ko.doc2bow(text) for text in texts_promises]
# Train Lsi model
lsi = models.LsiModel(corpus, id2word=dictionary_ko, num_topics=num_topics)
# Transform corpus to LSI space and index it
lsi_corpus = lsi[corpus]
index = similarities.MatrixSimilarity(lsi_corpus)

CPU times: user 542 ms, sys: 63.6 ms, total: 606 ms
Wall time: 617 ms


In [111]:
def find_similar(tagged_text, similarity_threshold=None):
    # Convert text to bag of words
    vec_bow = dictionary_ko.doc2bow(tagged_text)
    # Convert the query to LSI space
    vec_lsi = lsi[vec_bow]
    # Perform a similarity query against the corpus
    sims = enumerate(index[vec_lsi])
    if similarity_threshold:
        sims = filter(lambda item: item[1] >= similarity_threshold, sims)
    # Return the similarities sorted by descending score
    return sorted(sims, key=lambda item: -item[1])
    
# Try to match a few existing documents

for text in texts[100:101]:
    sims = find_similar(text[1], similarity_threshold=0.75)
    print("Most similar to %s (%s)" % (text[0], ' '.join(text[1])))
    for key, score in sims[1:]:
        key = int(key)
        print('%.2f - %s' % (score, texts[key][0]))


Most similar to 동북아 대도시 대기환경협의체 구성 및 대기개선 노하우 공유 (동북아/Noun 대도시/Noun 대기/Noun 환경/Noun 협의/Noun 체/Noun 구성/Noun 대기/Noun 노하우/Noun 공유/Noun)
1.00 - 교육지원협의체 구성
0.98 - 동대문구 일자리창출정책협의회 구성
0.97 - 4) 복지 네트워크 구성 및 사회적 연대틀 강화
0.96 - 도시 아카데미 구성
0.95 - 지방의회 산하 생활안정센터 구성
0.92 - 구룡마을 문제 해결을 위한 전문가협의체 구성 및 합리적 방안 모색
0.92 - 다문화 가정 지원을 위한 전담팀 구성 · 운영
0.84 - 강북구 학교폭력예방지원협의회 설치
0.84 - 생활구정 혁신본부 구성
0.75 - 서울-세종 간 고속도로 강동통과 반대 지속 협의


In [112]:
%%time
import numpy as np
# Find similarities of all documents with each other and see how many matches higher than x score there are
# Used to figure out a threshold that regards enough documents as matched
threshold = 0.75
counts = []
counts_self_no_match = []
for idx, text in enumerate(texts[:500]):
    similarities = np.array(find_similar(text[1]))
    # Remove self-similarity
    id_idx = np.where(similarities[:, 0]==idx)[0][0]
    similarities = np.delete(similarities, id_idx, axis=0)
    # Record when self wasn't the best match (it is usally the 2nd best in these cases)
    if id_idx != 0:
        counts_self_no_match.append((idx, id_idx))
    # Count number of docs with similarity >= threshold
    count = np.sum(similarities[:,1]>=threshold)
    counts.append(count)

counts = np.array(counts)
counts_self_no_match = np.array(counts_self_no_match)
print('Number of matches per document: max %d, min %d, mean %.2f, median %d' % (np.max(counts), np.min(counts), np.mean(counts), np.median(counts)))
print('%d documents didn\'t match with themself as their best match' % (len(counts_self_no_match)))
if len(counts_self_no_match):
    print('  but %d of them matched at pos 1.' % (np.sum(counts_self_no_match[:,1]==1)))

no_match_count = np.sum(counts==0)
print('%d (%d%%) with 0 matches' % (no_match_count, 100*no_match_count/len(counts)))

Number of matches per document: max 26, min 0, mean 8.45, median 8
7 documents didn't match with themself as their best match
  but 2 of them matched at pos 1.
56 (11%) with 0 matches
CPU times: user 1.12 s, sys: 19.8 ms, total: 1.14 s
Wall time: 1.21 s


In [113]:
print("Document with highest number of matches:")
print(texts[np.argmax(counts)])

Document with highest number of matches:
('장애인회관 건립', ['장애인/Noun', '회관/Noun', '건립/Noun'], ('seoul', 3))


In [114]:
print("Unmatched documents:")
for idx in np.where(counts==0)[0]:
    print(texts[idx][0], texts[idx][1])

Unmatched documents:
어린이 보호구역 내 교통사고 ZERO 목표로 종합대책 추진 ['어린이/Noun', '보호/Noun', '구역/Noun', '내/Noun', '교통사고/Noun', 'ZERO/Alpha', '목표/Noun', '종합/Noun', '대책/Noun']
동대문 창조경제 클러스터 육성 ['동대문/Noun', '창조경제/Noun', '클러스터/Noun']
공공서비스 중심 맞춤형 100대 적합업종 발굴 ['공공/Noun', '서비스/Noun', '중심/Noun', '맞춤/Noun', '대다/Verb', '적합/Noun', '업종/Noun', '발굴/Noun']
자영업 지원센터’ 설치 ['자/Noun', '영업/Noun', '센터/Noun']
성곽마을 재생프로젝트를 통한 유럽형 명품주거단지 조성 ['성곽/Noun', '마을/Noun', '재생/Noun', '프로젝트/Noun', '통한/Noun', '유럽/Noun', '명품/Noun', '주거/Noun', '단지/Noun']
서울형 도시재생 1조원 투자 ['서울/Noun', '도시재생/Noun', '조원/Noun', '투자/Noun']
서울형인증제 확대로 요양서비스 질 제고 ['서울/Noun', '인증/Noun', '제/Noun', '요양/Noun', '서비스/Noun', '질/Noun', '제/Noun']
50+ 사회공헌아카데미 교육 확대 ['사회/Noun', '공헌/Noun', '아카데미/Noun', '교육/Noun']
50+ 경험과 노하우를 배우는 온라인 인생학교(TED) 조성 ['경험/Noun', '노하우/Noun', '배우다/Verb', '온라인/Noun', '인생/Noun', '학교/Noun', 'TED/Alpha']
돌봄 주부를 위한 주부휴가제 시행 ['돌보다/Verb', '주부/Noun', '위/Noun', '하다/Verb', '주부/Noun', '휴가/Noun', '제/Noun', '시행/Noun']
<안심귀가 스카우트> 확대 운영 ['안심/Noun', '귀가/Noun',

In [116]:
# Try some queries that don't match existing documents

def match(title):
    tags = pos(title)
    print("Most similar to %s (%s)" % (title, " ".join(tags)))
    threshold = 0.75
    for posi, item in enumerate(find_similar(tags, threshold)):
        key = int(item[0])
        score = item[1]
        print('%2d. %.2f - %s - %s %d' % (posi+1, score, texts[key][0], texts[key][2][0], texts[key][2][1]))

match('임대주택')
match(' 임대주택 등록하면 세금·건보료 감면…2020년 등록 의무화 검토')
match('지난해 저소득층 근로소득 감소에…소득 불평등도 악화 : 네이버 뉴스')
match('노후자금 털어 집에 투자? 전세금, 집값 용도 퇴직연금 중도인출 급증 : 네이버 뉴스')

text = "대학 등록금 동결 속 외국인 유학생만 '봉' 서울 주요 대학들이 외국인 유학생이 내야 할 등록금을 잇달아 올리려 하는 것으로 드러났다. 내국인 대학생에 대한 ‘등록금 동결’과 ‘입학금 폐지’로 줄어든 수입을 외국인 유학생에게서 더 받아내 만회하겠다는 것이다."
match(text)

Most similar to 임대주택 (임대주택/Noun)
 1. 0.98 - 8) 대학생 임대주택 공급 지원 - seoul 17
 2. 0.96 - 장기 공공임대주택 2500호 공급 -> 장기 공공임대주택 2,400호 공급 - seoul 16
 3. 0.91 - 태양광 설치비’ 장기 저금리 융자지원 - seoul 0
 4. 0.90 - 스마트형 공공 쓰레기통, ‘진격의 스마통’ - seoul 0
 5. 0.90 - 공공보육시설 확충 - seoul 1
 6. 0.88 - 공공청사 태양광발전소 건립 확대 - seoul 2
 7. 0.88 - 공공청사 장소대관 활성화 - seoul 2
 8. 0.87 - 맞벌이 부부를 위한 역세권 공공보육시설 설치 - seoul 13
 9. 0.86 - 공공·작은도서관 지속적 확충 및 운영 개선 - seoul 0
10. 0.80 - 공공부문 채용시 전체의 3%는 은퇴자, 노인 고용 의무화 추진 - seoul 16
11. 0.79 - 365일 24시간 운영 공공보육시설 시범 설치 - seoul 1
12. 0.76 - 공공 노인요양원 30개소 설치 - seoul 0
13. 0.76 - 세대융합(화합)형 공공주택 공급 - seoul 0
Most similar to  임대주택 등록하면 세금·건보료 감면…2020년 등록 의무화 검토 (임대주택/Noun 등록/Noun 하다/Verb 세금/Noun 건/Noun 보료/Noun 감다/Verb 등록/Noun 의무/Noun)
Most similar to 지난해 저소득층 근로소득 감소에…소득 불평등도 악화 : 네이버 뉴스 (지난해/Noun 소득/Noun 층/Noun 근로/Noun 소득/Noun 감소/Noun 소득/Noun 불평등/Noun 악화/Noun)
 1. 1.00 - 저소득층 지원 확충 - seoul 3
 2. 1.00 - 무상보육을 소득하위 80%까지 확대 추진 - seoul 1
 3. 1.00 - 저소득층 자녀 지원프로그램 강화 - seoul 2
 4. 1.00 - 저소득층 자녀 수업료 · 등

In [117]:
corpora.MmCorpus.serialize('lsi.mm', lsi_corpus)

In [118]:
%%time
import numpy as np
from sklearn.manifold import TSNE
from gensim import matutils

lsi_corpus = corpora.MmCorpus('lsi.mm')
tsne = TSNE(n_components=2, init='pca', random_state=0, metric='cosine', angle=.99)
X = matutils.corpus2dense(lsi_corpus, num_terms=lsi_corpus.num_terms)
X_data = np.asarray(X).astype('float64')
X_tsne = tsne.fit_transform(X_data.T)

CPU times: user 21.1 s, sys: 2.88 s, total: 24 s
Wall time: 25.2 s


In [119]:
threshold = 0.5
_idx = np.amax(X, axis=0) > threshold
X_ = X[:, _idx]
X_tsne_ = X_tsne[_idx, :]

In [120]:
#X_tsne[_idx, 0].shape
X_tsne_.shape

(306, 2)

In [123]:
import matplotlib
matplotlib.rc('font', family='NanumGothic')

from matplotlib import pyplot as plt
top_topics = np.argmax(X_, axis=0)

topics_words = lsi.show_topics(num_words=5, formatted=False)
topic_summaries = [' '.join([word[0].split('/')[0] for word in topic]) for topic_no, topic in topics_words]

plt.figure(figsize=(15,10))
plt.scatter(X_tsne_[:, 0], X_tsne_[:, 1], c=top_topics, cmap=plt.cm.get_cmap("jet", 50))

plt.colorbar(ticks=range(50))
plt.clim(-0.5, 9.5)

# plot topic summaries
topic_coord = np.empty((X_.shape[1], 2)) * np.nan
for topic_num in top_topics:
    if not np.isnan(topic_coord).any():
        break
    topic_coord[topic_num] = X_tsne_[np.argmax(top_topics==topic_num)]

# plot crucial words
for i in range(len(topic_summaries)):
    if np.isnan(topic_coord[i]).any():
        continue
    plt.text(topic_coord[i, 0], topic_coord[i, 1], topic_summaries[i])


#plt.show()
plt.savefig('tsne.png')

In [124]:
import numpy as np
import bokeh.plotting as bp
from bokeh.plotting import output_file, save
from bokeh.models import HoverTool
from random import shuffle

colors = ["#E495A5", "#E3979D", "#E19895", "#DE9A8E", "#DB9C86", "#D89F7F", "#D3A178", "#CEA372", "#C9A66D", "#C3A869", "#BDAB66", "#B6AD65", "#AFAF64", "#A7B166", "#9EB368", "#96B56C", "#8DB771", "#83B977", "#79BA7E", "#6FBB84", "#65BC8C", "#5ABD93", "#50BE9B", "#46BEA2", "#3FBEAA", "#39BEB1", "#37BDB8", "#3ABCBF", "#41BBC5", "#4ABACB", "#55B8D0", "#61B6D5", "#6DB4D9", "#79B1DC", "#85AEDF", "#91ACE1", "#9CA9E2", "#A6A6E2", "#B0A3E1", "#B9A0E0", "#C29DDE", "#C99ADB", "#D098D7", "#D596D3", "#DA95CD", "#DE94C8", "#E093C1", "#E393BB", "#E493B4", "#E494AC"]
shuffle(colors)
colormap = np.array(colors)
title = 'Promises LSI vizualization'
num_example = len(X_)

plot = bp.figure(plot_width=1200, plot_height=700,
                     title=title,
                     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot.scatter(x='x', y='y',
                 color='color',
                 source=bp.ColumnDataSource({
                   "x": X_tsne_[:, 0], "y": X_tsne_[:, 1],
                   "content": np.array([text[0] for text in texts])[_idx],
                   "topic_key": top_topics,
                   "color": colormap[top_topics]
                   }))

for i in range(len(topic_summaries)):
    if np.isnan(topic_coord[i]).any():
        continue
    plot.text(topic_coord[i, 0], topic_coord[i, 1], [topic_summaries[i]])

    
hover = plot.select(dict(type=HoverTool))
hover.tooltips = {"content": "@content - topic: @topic_key"}
output_file('{}.html'.format(title))
save(plot)


'/Users/paul/Projects/kixlab/xproject/xproj-utils/promise-match/Promises LSI vizualization.html'